In [ ]:
# ***************************************************************
# Preprocess Microsoft Academics Graph (MAG) Dataset
# ***************************************************************

# Jupyter Notebook for the preprocessing of the Microsoft Academics Graph (MAG) dump
#
# TODO **********
# In particular, the following operations are going to be executed:
# - (sequential) opening of the dump's CSVs
# - Drop of the useless columns (only 'cited' is going to remain)
# - Gruop by on the 'cited' column and count
# - Union with the dataframe of the CSVs previously elaborated
#
# Lastly, the entire preprocessed dump is going to be saved on disk in CSV format

In [ ]:
# Libraries Import
import pandas as pd
import glob

In [ ]:
# ******************* PATHS ********************+

# Dumps Directory Path
path_file_import = r'/Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/MAG/'

# CSV Exports Directory Path
path_file_export = r'/Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Export/'

# Combine New Data with a "Partial" CSV
#
# This can be really useful in case of limited disk space, allowing us to partially process 
# the dump (using a subset of the CSVs) and free some space on disk by deleting the CSVs 
# that have been already processed.
#
# Note: the delete operations need to be made manually
# Note: the partial CSV needs to be in the same format of the one generated with this script
combine_with_partial_csv = False
partial_csv_path = r'/Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Export/out_coci_partial.csv'

In [ ]:
# Get All Files' Names
coci_all_csvs = glob.glob(path_file_import + "*.csv")

In [ ]:
df_coci_processed = pd.DataFrame(columns=['article', 'citations_count'])

# Combine new data with a partial CSV
if combine_with_partial_csv:
    df_coci_processed = pd.read_csv(partial_csv_path, low_memory=False)
    print(f'Successfully Imported the Partial CSV')
    
# Read, process and concat all CSVs
count = 0
for current_csv_name in coci_all_csvs:

    # Open the current CSV
    print(f'Currently processing CSV {count}: {current_csv_name}')
    count += 1
    df_coci_current_csv = pd.read_csv(current_csv_name, low_memory=False)

    # Drop of the useless columns: 'oci', 'citing', 'creation', 'timespan', 'journal_sc', 'author_sc'
    df_coci_current_csv.drop(columns=['oci', 'citing', 'creation', 'timespan', 'journal_sc', 'author_sc'])

    # Group by cited article and count
    sf_coci_current_grouped = df_coci_current_csv.groupby(['cited'])['cited'].count()

    # Since the returned object is a Pandas Series type, we need to convert it to a Pandas dataframe
    df_coci_current_csv = pd.DataFrame({'article':sf_coci_current_grouped.index, 'citations_count':sf_coci_current_grouped.values})

    ### Concat with the data previously elaborated
    df_coci_processed = pd.concat([df_coci_processed, df_coci_current_csv])

    # Now we need to do a new group by and sum the citations_count to reduce the data
    sf_coci_processed_grouped = df_coci_processed.groupby(['article'])['citations_count'].sum()
    df_coci_processed = pd.DataFrame({'article':sf_coci_processed_grouped.index, 'citations_count':sf_coci_processed_grouped.values})

# Export of the final dataframe
df_coci_processed.to_csv(path_file_export + 'out_coci_citations_count.csv')
print(f'Successfully Exported the Preprocessed CSV to {path_file_export}out_coci_citations_count.csv')


In [ ]:
# Check of the Exported CSV
df_coci_exported_csv = pd.read_csv(path_file_export + 'out_coci_citations_count.csv', low_memory=False)
df_coci_exported_csv

In [ ]:
# Order by citations count descending to see the articles with the most citations
df_coci_exported_csv = df_coci_exported_csv.sort_values(by='citations_count', ascending=False)
df_coci_exported_csv

In [ ]:
# Checking the total count of the citations contained in the extracted CSV
df_coci_exported_csv['citations_count'].sum()